In [37]:
import numpy as np
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit import BasicAer, execute

## Circuit Creation

### Preperation
Prepares a Bell-state by performing ($H\otimes I$) on the first qubit and entangling it with the second qubit.

In [38]:
encodeQ1 = QuantumRegister(1, 'encodeQ1')
encodeQ2 = QuantumRegister(1, 'encodeQ2')

preperationCircuit = QuantumCircuit(encodeQ1, encodeQ2)
preperationCircuit.h(encodeQ1)
preperationCircuit.cx(encodeQ1, encodeQ2)
preperationCircuit.draw()

### Encode
Encodes the message into the first Qubit. This is done via a controlled $X$ (NOT) Gate and $Z$ (Phase) Gate based on the message. This rotation changes the Bell-state such that when the qubit is sent, it can be recombined with the other encoding qubit via a controlled $X$ Gate to retrieve the message. 

In [39]:
encodedMessage = ClassicalRegister(2, 'em')
messageQubits = QuantumRegister(2, 'cm')

encodeCircuit = QuantumCircuit(messageQubits, encodeQ1)

# Encoding
encodeCircuit.cx(messageQubits[1], encodeQ1)
encodeCircuit.cz(messageQubits[0], encodeQ1)
encodeCircuit.draw()

### Input

In [40]:
# Get sent classical message
inputCircuit = QuantumCircuit(messageQubits, encodedMessage)
inputCircuit.measure(messageQubits, encodedMessage)
inputCircuit.draw()

### Transfer
This swap operation represents a transfer of quantum information from one information system to another.

In [41]:
swapQubit = QuantumRegister(1, 'swap')

transferCircuit = QuantumCircuit(encodeQ1, swapQubit)
transferCircuit.swap(encodeQ1, swapQubit)
transferCircuit.draw()

### Decode
Decode the message using the prepared qubit and the newly arrived qubit.

In [42]:
decodeCircuit = QuantumCircuit(swapQubit, encodeQ2)
decodeCircuit.cx(swapQubit, encodeQ2)
decodeCircuit.h(swapQubit)
decodeCircuit.draw()

In [43]:
receivingRegisters = ClassicalRegister(2, 'recreg')

readCircuit = QuantumCircuit(swapQubit, encodeQ2, receivingRegisters)
readCircuit.measure(swapQubit[0], receivingRegisters[0])
readCircuit.measure(encodeQ2[0], receivingRegisters[1])
readCircuit.draw()

In [44]:
allCircuits = (preperationCircuit + inputCircuit + encodeCircuit + transferCircuit + decodeCircuit + readCircuit)

def setupCircuit(message, messageQubits):
    circuit = QuantumCircuit(messageQubits)
    if message[0] is '1':
        circuit.x(messageQubits[1])
    if message[1] is '1':
        circuit.x(messageQubits[0])
    return circuit

## Testing

Test that the encoding circuit and the decoding circuit are decoupled from each other using num_tensor_factors to determine the number of subcircuits.

In [45]:
quantum_systems_count = 2
try:
    assert ((encodeCircuit + (decodeCircuit + readCircuit)).num_tensor_factors() == quantum_systems_count)
    print("Encode circuit and decode circuit are decoupled.")
except:
    print("Circuits overlap!")

Encode circuit and decode circuit are decoupled.


In [51]:
backend_sim = BasicAer.get_backend('qasm_simulator')
for case in ['00', '01', '10', '11']:
    circuit = (setupCircuit(case, messageQubits) + preperationCircuit + inputCircuit + encodeCircuit + transferCircuit + decodeCircuit + readCircuit)

    result = execute(circuit, backend_sim, shots=1).result()
    counts = result.get_counts(circuit)
    try:
        assert (counts[f"{case} {case}"] == 1)
        print (f"{case} was sent successfully!")
    except:
        print (f"{case} got result {counts}")

00 was sent successfully!
01 was sent successfully!
10 was sent successfully!
11 was sent successfully!
